In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Reshape, Activation, Dropout

# Input
input_1 = Input(shape=(None, 25, 25, 1))
input_2 = Input(shape=(None, 25, 25, 1))

# Reshape (25, 25, 1) to (25*25, 1)
reshaped_input_1 = Reshape((-1, 25 * 25))(input_1)
reshaped_input_2 = Reshape((-1, 25 * 25))(input_2)

# Concatenate inputs along the last axis
merged = tf.concat([reshaped_input_1, reshaped_input_2], axis=-1)  # Shape: (batch_size, time_steps, 25*25*2)

# LSTM Layer without Dropout
lstm_out = LSTM(units=64, return_sequences=True, recurrent_activation='sigmoid')(merged)

# Dropout after LSTM
lstm_out = Dropout(0.2)(lstm_out)

# 50 Units: 2 Classification, Each 25 options
dense_out = Dense(units=50)(lstm_out)

# Dropout after Dense
dense_out = Dropout(0.2)(dense_out)

# Shape: (batch_size, time_steps, 2, 25)
reshaped_out = Reshape((-1, 2, 25))(dense_out)

# Apply softmax activation
output = Activation('softmax')(reshaped_out)

# Create the model
model = Model(inputs=[input_1, input_2], outputs=output)

def cancel_loss(y_true, y_pred):
    y_true_truncated = y_true[:, 40:, :, :]
    y_pred_truncated = y_pred[:, 40:, :, :]
    loss = tf.keras.losses.categorical_crossentropy(y_true_truncated, y_pred_truncated)
    return tf.reduce_mean(loss)

model.compile(optimizer='adam', loss=cancel_loss)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 input_4 (InputLayer)           [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 reshape_3 (Reshape)            (None, None, 625)    0           ['input_3[0][0]']                
                                                                                            

In [2]:
!pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [3]:
import os
import numpy as np
import tensorflow as tf

def data_generator(file_list, data_dir, batch_size):
    total_files = len(file_list)
    
    while True:
        np.random.shuffle(file_list)
        for i in range(0, total_files, batch_size):
            batch_files = file_list[i:i + batch_size]
            feat_batch, army_batch, y_batch = [], [], []

            for file in batch_files:
                file_path = os.path.join(data_dir, file)
                
                data = np.load(file_path, allow_pickle=True)
                
                feat_batch.append(data['feat_padded'].astype(np.float32))
                army_batch.append(data['army_padded'].astype(np.float32))
                Y_padded = data['Y_padded'].astype(np.float32)
                
                # print(np.max(Y_padded[:,0]),np.max(Y_padded[:,1]))
                y_dim1_one_hot = tf.keras.utils.to_categorical(Y_padded[:, 0], num_classes=25)
                y_dim2_one_hot = tf.keras.utils.to_categorical(Y_padded[:, 1], num_classes=25)
                y_one_hot = np.stack([y_dim1_one_hot, y_dim2_one_hot], axis=1)
                y_batch.append(y_one_hot.astype(np.float32))
            
            feat_batch = np.array(feat_batch)
            army_batch = np.array(army_batch)
            y_batch = np.array(y_batch)
            # print(feat_batch.shape)
            yield (feat_batch, army_batch), y_batch


In [4]:
import os
from sklearn.model_selection import train_test_split

data_dir = "autodl-tmp/Numpy_Data"
file_names = os.listdir(data_dir)

train_files, test_files = train_test_split(file_names, test_size=0.2, random_state=42)  # 80% train, 20% test
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 75% train, 25% val

In [5]:
print(len(train_files),len(val_files),len(test_files))

2615 872 872


In [6]:
file_path = '/root/autodl-tmp/Numpy_Data/data_row_16.npz'
data = np.load(file_path, allow_pickle=True)
data_keys = data.files
print(data_keys)
# print(data['Y_padded'].shape)
Y_padded = data['Y_padded']
print(Y_padded.shape)
print("Max value in Y_padded[:, 1]:", np.max(Y_padded[:, 0]))

['feat_padded', 'army_padded', 'Y_padded']
(1000, 2)
Max value in Y_padded[:, 1]: 5


In [7]:
batch_size = 16

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
train_dataset = train_dataset.shuffle(buffer_size=100)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
# val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
# test_dataset = test_dataset.batch(batch_size)

In [8]:
for batch in train_dataset.take(1):  # Inspect 1 batch
    (feat_padded, army_padded), y_padded = batch
    print("feat_padded shape:", feat_padded.shape)
    print("army_padded shape:", army_padded.shape)
    print("y_padded shape:", y_padded.shape)

2024-09-21 10:36:42.747761: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 26 of 100
2024-09-21 10:36:52.724021: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 53 of 100
2024-09-21 10:37:02.677198: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 80 of 100


feat_padded shape: (16, 1000, 25, 25)
army_padded shape: (16, 1000, 25, 25)
y_padded shape: (16, 1000, 2, 25)


2024-09-21 10:37:10.259974: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


In [9]:
print(train_dataset)


<ShuffleDataset element_spec=((TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32, name=None)), TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32, name=None))>


In [12]:
from tensorflow.keras.callbacks import EarlyStopping

total_files = len(os.listdir("autodl-tmp/Numpy_Data"))
train_steps = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(train_dataset, 
          epochs=100, 
          steps_per_epoch=train_steps, 
          validation_data=val_dataset,
          validation_steps=validation_steps,
          callbacks=[early_stopping])

model.save('model_full.h5')

Epoch 1/100


2024-09-21 10:41:58.239016: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 25 of 100
2024-09-21 10:42:08.173856: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 55 of 100
2024-09-21 10:42:18.252031: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 85 of 100
2024-09-21 10:42:23.306690: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2024-09-21 10:42:24.149292: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


163/163 [==============================] - 128s 553ms/step - loss: 1.2762 - val_loss: 0.7810
Epoch 2/100
163/163 [==============================] - 90s 553ms/step - loss: 1.0584 - val_loss: 0.7625
Epoch 3/100
163/163 [==============================] - 92s 564ms/step - loss: 1.0285 - val_loss: 0.7567
Epoch 4/100
163/163 [==============================] - 93s 571ms/step - loss: 0.9973 - val_loss: 0.7542
Epoch 5/100
163/163 [==============================] - 91s 556ms/step - loss: 0.9757 - val_loss: 0.7515
Epoch 6/100
163/163 [==============================] - 90s 551ms/step - loss: 0.9495 - val_loss: 0.7588
Epoch 7/100
163/163 [==============================] - 90s 550ms/step - loss: 0.9163 - val_loss: 0.7563
Epoch 8/100
163/163 [==============================] - 90s 553ms/step - loss: 0.9012 - val_loss: 0.7636
Epoch 9/100
163/163 [==============================] - 93s 572ms/step - loss: 0.8804 - val_loss: 0.7634
Epoch 10/100
163/163 [==============================] - 91s 560ms/step - lo

NameError: name 'test_generator' is not defined

In [14]:
test_steps = len(test_files) // batch_size
model.evaluate(test_dataset, steps=test_steps)

54/54 [==============================] - 22s 402ms/step - loss: 0.7873


0.7872810363769531

In [ ]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    feat_padded, army_padded, Y_padded, test_size=0.2, random_state=42
)

In [ ]:
del feat_padded, army_padded, Y_padded
gc.collect()

In [ ]:
y_train = np.squeeze(y_train, axis=2)
y_test = np.squeeze(y_test, axis=2)

In [ ]:
num_classes = 25

y_train_class_1 = tf.one_hot(y_train[..., 0], depth=num_classes)
y_train_class_2 = tf.one_hot(y_train[..., 1], depth=num_classes)
y_train_categorical = tf.stack([y_train_class_1, y_train_class_2], axis=-2)

y_test_class_1 = tf.one_hot(y_test[..., 0], depth=num_classes)
y_test_class_2 = tf.one_hot(y_test[..., 1], depth=num_classes)
y_test_categorical = tf.stack([y_test_class_1, y_test_class_2], axis=-2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit([X1_train, X2_train], y_train_categorical, epochs=100, batch_size=16, verbose=1)
model.save('model_full.h5')
model.evaluate([X1_test, X2_test], y_test_categorical, verbose=1)